In [1]:
#Version Python utilisée
import sys
print(sys.version)

3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]


In [213]:
#Cette cellule permet l'extraction des données du ficheir GoogleSheet Excel 
#      DONC les PAYS
#Et de stocker sous forme de fichier en local sur un poste Windows
# import des library
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import urllib.parse
import re

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
#SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly','https://spreadsheets.google.com/feeds',
#'https://www.googleapis.com/auth/drive']
# récupération du fichier
#URL de partage du googlesheet
#https://docs.google.com/spreadsheets/d/1rbJoVxUbq9lAxpO4zdZex2-2QaS5Zq7-Jh9b0ceRMGA/edit?usp=sharing
#URL du googlesheet : https://docs.google.com/spreadsheets/d/1rbJoVxUbq9lAxpO4zdZex2-2QaS5Zq7-Jh9b0ceRMGA/edit?ts=609d10ec#gid=425812502
SAMPLE_SPREADSHEET_ID = '1rbJoVxUbq9lAxpO4zdZex2-2QaS5Zq7-Jh9b0ceRMGA'
SPREADSHEET_ID = '1rbJoVxUbq9lAxpO4zdZex2-2QaS5Zq7-Jh9b0ceRMGA'
urlParts = ''

def token():
    """Mise en place d'une API avec identification des utilisateurs et droits préalable
    URL de gestion des droits : 
    https://console.cloud.google.com/apis/credentials/consent?folder=&organizationId=&project=rosy-hope-317409
    """
    creds = None
    # Le fichier token.json enregistre l'accès utilisateur & MAJ du jeton(token)
    # Création automatique quand le circuit d'authentiication est réalisé pour la premiere fois
    if os.path.exists('python.json'):
        creds = Credentials.from_authorized_user_file('python.json', SCOPES)
    # S'il n'y a pas de "credential" valable (valide) alors il faut demander à l'utilisateur de s'authentifier.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            #creds.refresh(Request())
        #else:
            flow = InstalledAppFlow.from_client_secrets_file('client_secret_1011095565577-fbe0c7g9f6mo7psasu2kjs799dequ5e9.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Enregistrement du "credentials" pour le prochain lancement
        with open('python.json', 'w') as token:
            token.write(creds.to_json())
    return creds

#Récupération de l'ensemble des données du googlesheet
def sheet(creds):
    #Création du service
    service = build('sheets', 'v4', credentials=creds)
    #Création de l'objet contenant les données de la page google sheet
    sheet_metadata = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
    #Proprietés de la page
    properties = sheet_metadata.get('sheets')
    #initialisation de la liste
    sheetNames=list()
    #Boucle pour récupération des titres des pages,de leur identifiants et de leur URL dans un tableau
    for item in properties:
        sheetNames.append([item.get("properties").get('title'),item.get("properties").get('sheetId')])
    return(sheetNames)

#Suppression des virgules et des caractères de fin de ligne dans la cellule (saut de lignes, espaces)
def remove_comma(string):
    string = string.replace(",","/")
    string = string.rstrip()
    return string

#Création d'un dataFrame a partir d'une liste en dur des colonnes contrainte SendInBlue
def createDF(myListe):
    dfData = pd.DataFrame(myListe, columns=["SITE_MEDIA","TYPE","MEDIA","PERIODICITE","EMISSION","SEXE","NOM","PRENOM","POSTE","DOMAINE","TELEPHONE","EMAIL","PAYS","FAUX A CHANGER"])
    return(dfData)

#Sauvegarde du fichier en local attention a revoir lors de la mise en palce sur une serveur
def save(dfData,fichier) : 
    Path = os.path.abspath(f'Documents/{fichier}.csv')
    dfData.to_csv(Path,index=False)    

#nombre d'éléments par ligne (14)
def clean(sheetNames, creds):
    service = build('sheets', 'v4', credentials=creds)
    dataPays=[]
    for onglet in sheetNames:
        Path = os.path.abspath(f'Documents/{onglet[0]}.csv')
        dictPaysOnglet = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET_ID, range=urllib.parse.unquote(onglet[0])).execute()
        #Exclusion des feuilles vides
        if 'values' in dictPaysOnglet :
            ongletDataPays=[]
            #Exclusion de la première feuille de calcul 'A LIRE'
            if 'A LIRE' not in dictPaysOnglet['range'] :
                ##Data except first sheet
                #print(dictPaysOnglet['values'][1:])
                #problème des virgules dans les cellules
                #suppression des virgules dans les éléments de la liste
                #dataPaysOnglet = [remove_comma(i) for i in dictPaysOnglet['values'][1:]]
                #dictPaysOnglet['values'][1:]
                for nbMot in dictPaysOnglet['values'][1:] :
                    while(len(nbMot))<14 :
                        nbMot.append('NONE')
                    ligneDataPaysOnglet=[]
                    for mot in nbMot :
                        ligneDataPaysOnglet.append(remove_comma(mot))
                    #Enregistrement onglet par onglet
                    ongletDataPays.append(ligneDataPaysOnglet)
                    #Enregistrement de l'ensemble des données onglet par onglet 
                    dataPays.append(ligneDataPaysOnglet)
                    #Création du Dataframe
                    dfData = createDF(ongletDataPays)
                    #Enregistrement 
                    repPays = 'Pays/'+onglet[0]
                    save(dfData,repPays)
    dfComplet = createDF(dataPays)
    dfComplet.dtypes
    save(dfComplet,"Pays/DATA Pays")
            #save(dataPays, "dataPays")
            ##Columns
            #print(dictPaysOnglet['values'][0][0:len(dictPaysOnglet['values'][0])-1])
            #print(tableauDataPaysOnglet)
            #dfDataPays = pd.DataFrame(dataPays, columns=dictPaysOnglet['values'][0][0:len(dictPaysOnglet['values'][0])-1])
            #dfDataPays.head()
                #read_csv ne fonctionne pas a cause des données au format dict
                #df = pd.read_csv(dictPaysOnglet['values'][1:], index_col=dictPaysOnglet['values'][0][0:len(dictPaysOnglet['values'][0])-2], sep=',')
                #dataframe ne fonctionne pas a cause du doublequote quand la donnée contient une simple quote
                #Permet de remplacer les , entre les '
                #re.sub(r'"(\d+),(\d+)"', r'\1:\2', line)
                #re.sub(r'(?!(([^"]*"){2})*[^"]*$),', "", input);
                #print(type(dictPaysOnglet)) == Dict
                #print(data)
                #data = re.sub(r'(?!(([^"]*"){2})*[^"]*$),'," ",dictPaysOnglet['values'][1:])
                #dfDataPaysOnglet = pd.DataFrame(re.sub(r'(?!(([^"]*"){2})*[^"]*$),'," ",dictPaysOnglet['values'][1:]), columns=dictPaysOnglet['values'][0][0:len(dictPaysOnglet['values'][0])-2])
                #dfDataPaysOnglet.head()
                ##Impossible de gerer les virgule non plu
                #dfDictOnglet = pd.DataFrame.from_records(dictPaysOnglet['values'][1:],columns=dictPaysOnglet['values'][0][0:len(dictPaysOnglet['values'][0])-2])
                #dfDictOnglet.head()
                #print(dictPaysOnglet['values'][0])
                #print(dictPaysOnglet)
            #dictPaysOnglet.to_csv(Path,index=False)   

#def groupBy (dfcomplet) :
    
if __name__ == '__main__':
    creds = token()
    sheetNames = sheet(creds)
    clean(sheetNames,creds)

In [217]:
#Cette cellule permet d'extraire du fichier complet les domaines
#DOMAINES

import pandas as pd
import numpy as np
import os.path

#Sauvegarde du fichier en local attention a revoir lors de la mise en place sur une serveur
def save(dfData,fichier) : 
    Path = os.path.abspath(f'Documents/{fichier}.csv')
    dfData.to_csv(Path,index=False) 
    

#lecture du fichier CSV
dfComplet = pd.read_csv(f'Documents/Pays/DATA Pays.csv',header=0,index_col=False)
#Attention il ne faut pas récupérer les multi domaines qui contienent "/"
dfDomN = dfComplet.drop(index=dfComplet[dfComplet['DOMAINE'].str.contains('/', na=False)].index, inplace=False)
#Recuperation des <> valeur de domaine
#print(dfDomN)
#distinct
dfDomaines = dfDomN['DOMAINE'].drop_duplicates()
#suppression des valeur NONE 
dfDomaines = dfDomaines.replace(to_replace='NONE', value=np.nan).dropna()
#données par domaine
for domaine in dfDomaines :
    dfDataDomaine = dfComplet[dfComplet['DOMAINE'].str.contains(domaine, na=False)]
    save(dfDataDomaine,'Domaine/'+domaine)
#dfDataDomaine = dfComplet[dfComplet['Domaine'].str.contains(domaine, na=False)].index
#dfDomaines.head()
#file = domaine.replace("/","_")
#fichier = file.replace(" ","")
#Path = os.path.abspath(f'Documents/{fichier}.csv')
#dfDataDomaine.to_csv(Path,index=False)   

In [218]:
#SendInBlue
#Cette cellule reprend l'ensemble des fichiers locaux pour les importer dans SIB
from __future__ import print_function
import time
import sib_api_v3_sdk
import csv
from os import listdir
from sib_api_v3_sdk.rest import ApiException
from pprint import pprint
from unidecode import unidecode

#Configuration de l'accès a SIB
configuration = sib_api_v3_sdk.Configuration()
configuration.api_key['api-key'] = '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'

api_instance = sib_api_v3_sdk.ContactsApi(sib_api_v3_sdk.ApiClient(configuration))
#récpération depuis la date ci-dessous
#modified_since = '2020-01-20T19:20:30+01:00'
#Limite de la récupération à x
limit = 50
#offset = 0
def addFolder (monRep):
    #Ajouter un repertoire en dur
    api_instance = sib_api_v3_sdk.FoldersApi(sib_api_v3_sdk.ApiClient(configuration))
    #create_folder = sib_api_v3_sdk.CreateUpdateFolder(name=monRep)
    try:
        api_response = api_instance.create_folder(create_folder)
        return(api_response)
    except ApiException as e:
        print("Exception when calling FoldersApi->create_folder: %s\n" % e)

def recupFolder():
    #Récupération de tous les repertoires du projet de SenInBlue
    try:
        api_response = api_instance.get_folders(limit, offset)
        pprint(api_response)
    except ApiException as e:
        print("Exception when calling ContactsApi->get_folders: %s\n" % e)

def supprFolder (folder_id) : 
    #Delete Folder
    #folder_id = 197
    api_instance.delete_folder(folder_id)

def findFolder (folderId):
    api_instance = sib_api_v3_sdk.FoldersApi(sib_api_v3_sdk.ApiClient(configuration))
    #Récupération d'un repertoire spécifique
    try:
        api_response = api_instance.get_folder(folderId)
        return(api_response)
    except ApiException as e:
        print("Exception when calling FoldersApi->get_folder: %s\n" % e)

def creatTabl () :
    #Création d'un tableau id : liste associée
    tabIdListe = {}
    try:
        rep = api_instance.get_folder_lists(folder_id, limit=limit, offset=offset)
        for myRep in rep.lists :
            tabIdListe[myRep['id']] = myRep['name']
    except ApiException as e:
        print("Exception when calling ListsApi->get_lists: %s\n" % e)
    #Récupération des listes par id
    try:
        liste = api_instance.get_list(list_id)
        #pprint(liste)
    except ApiException as e:
        print("Exception when calling ListsApi->get_list: %s\n" % e)

def recupContactparListe (list_id) :
    #Récupération des contacts par liste (list_id)
    try:
        api_response = api_instance.get_contacts_from_list(list_id, modified_since=modified_since, limit=limit, offset=offset)
        pprint(api_response)
    except ApiException as e:
        print("Exception when calling ListsApi->get_contacts_from_list: %s\n" % e)

def recupListFromFolder(folderId):
    api_instance = sib_api_v3_sdk.FoldersApi(sib_api_v3_sdk.ApiClient(configuration))
    #folder_id = 1
    #limit = 10
    offset = 0

    try:
        api_response = api_instance.get_folder_lists(folderId, limit=limit, offset=offset)
        return (api_response)
    except ApiException as e:
        print("Exception when calling FoldersApi->get_folder_lists: %s\n" % e)

def getlistContact(listId):
    try:
        api_response = api_instance.get_contacts_from_list(listId, modified_since=modified_since, limit=limit, offset=offset)
        return(api_response)
    except ApiException as e:
        print("Exception when calling ListsApi->get_contacts_from_list: %s\n" % e)

def findPaysByName(monPays,monRep):
    #True/Flase est ce que le pays existe
    listPays = recupListFromFolder(monRep)
    for pays in listPays.lists :
        if unidecode(pays['name'].upper()) == unidecode(monPays.upper()) : 
            return(pays['id'])
    #return(pays['id'] for pays in listPays.lists if pays['name'] == monPays)

def createList(monPays) : 
    #folder_id c'est l'id du repertoire qui contient cette nouvelle liste 
    #Ici c'est toujours Pays = 28
    create_list = sib_api_v3_sdk.CreateList(name=monPays, folder_id=28)

    try:
        api_response = api_instance.create_list(create_list)
        return (api_response)
    except ApiException as e:
        print("Exception when calling ContactsApi->create_list: %s\n" % e)

def supprList(listId):
    try:
        api_instance.delete_list(listId)
    except ApiException as e:
        print("Exception when calling ListsApi->delete_list: %s\n" % e)

def importContact(fileDataCSV, listeId):
    request_contact_import = sib_api_v3_sdk.RequestContactImport()
    request_contact_import.file_body = fileDataCSV
    request_contact_import.list_ids = [listeId] 
    request_contact_import.email_blacklist = False
    request_contact_import.sms_blacklist = False
    request_contact_import.update_existing_contacts = True
    request_contact_import.empty_contacts_attributes = True
    pprint(request_contact_import)

    try:
        api_response = api_instance.import_contacts(request_contact_import)
        pprint(api_response)
    except ApiException as e:
        print("Exception when calling ContactsApi->import_contacts: %s\n" % e)

#Lecture du fichier
def monFichierImport(Path):
    return (os.path.exists(Path))

def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

def lectureImport(Path,libellePays,monRep):
#Gestion des exceptions
    #EXCEPTION = ['1_PANAFRICAIN FR','2_PANAFRICAIN AN','3_PANAFRICAIN AR','4_PANAFRICAIN LU']
    #20210830 mise en cohérence des libellés G excel/SIB
    EXCEPTION = []
    if ((os.path.exists(Path)) and (libellePays not in EXCEPTION)):
        #Récupération des données en local sur le poste windows pour import
        myFile = pd.read_csv(Path)
        #Formatage du fichier séparateur saut de ligne, sans index
        myFileCSV = "\n".join(myFile.to_csv(index=False).splitlines())
        #Rechercher si le pays existe dans SIB et si les données ont été extraites
        #Récupération de l'id du Pays
        idPays = findPaysByName(libellePays,monRep)
        #Récupération des données en local sur le poste windows pour import
        myFile = pd.read_csv(Path)
        #Formatage du fichier séparateur saut de ligne, sans index
        myFileCSV = "\n".join(myFile.to_csv(index=False).splitlines())
        #Si la liste n'existe pas et que les données ont été extraite
        if (idPays == None): 
            ##SI le pays n'existe pas alors création et import
            ##[28]=le repertoire pays peut etre récupérer mais pour l'instant pas d'interet
            myNewId = createList(libellePays)
            importContact(myFileCSV, myNewId.id)
            pprint("MAJ de {} N° Id : {} repertoire : {}".format(libellePays,myNewId,Path))
        #Si la liste existe dans SIB et que les données ont été extraites
        elif (idPays != None):
            ##Import des données au format CSV dans la liste SIB 
            #importContact(myFileCSV, idPays)
            pprint("MAJ de {}, N° Id : {} repertoire : {}".format(libellePays,idPays,Path))
    elif ((os.path.exists(Path)) and (onglet[0] in EXCEPTION)):
        #Récupération des données en local sur le poste windows pour import
        myFile = pd.read_csv(Path)
        #Formatage du fichier séparateur saut de ligne, sans index
        myFileCSV = "\n".join(myFile.to_csv(index=False).splitlines())
        if onglet[0] == '1_PANAFRICAIN FR' :
            Id = '56'
            #importContact(myFileCSV, idPays)
            pprint("MAJ de {} N° Id : {} repertoire : {}".format(onglet[0],Id,Path))
        elif onglet[0] == '2_PANAFRICAIN AN' :
            Id = '57'
            #importContact(myFileCSV, idPays)
            pprint("MAJ de {} N° Id : {} repertoire : {}".format(onglet[0],Id,Path))
        elif onglet[0] == '3_PANAFRICAIN AR' :
            Id = '58'
            #importContact(myFileCSV, idPays)
            pprint("MAJ de {} N° Id : {} repertoire : {}".format(onglet[0],Id,Path))
        elif onglet[0] == '4_PANAFRICAIN LU' :
            Id = '59'
            #importContact(myFileCSV, idPays)
            pprint("MAJ de {} N° Id : {} repertoire : {}".format(onglet[0],Id,Path))
    
    else:
        pprint("l'onglet {} du google sheet ne possède aucune données  ".format(onglet[0]))


if __name__ == '__main__':
    #Fonction utile pour comprendre l'organisation de SIB
    #Lecture des repertoires présent dans le projet
    #recupFolder()
    #Permet de voir les Listes + ID pour un repertoire : pays = 28 | Domaines = 29
    #pprint(recupListFromFolder(29))

    #type d'extension des fichiers
    extension='.csv'    

    #Boucle sur tous les pays
    #Lecture de tous les pays présent dans le googleSheet
    for onglet in sheetNames:
        monRep = 28
        #repertoire de stockage du fichier
        Path = os.path.abspath(f'Documents/Pays/'+onglet[0]+extension)
        lectureImport(Path,onglet[0],monRep)
    
    #Lecture de tous les domaines 
    for domaine in dfDomaines:
        print(domaine)
        monRep = 29
        #repertoire de stockage du fichier
        Path = os.path.abspath(f'Documents/Domaine/'+domaine+extension)
        lectureImport(Path,domaine,monRep)

('MAJ de FRANCOPHONE, N° Id : 56 repertoire : '
 'C:\\Users\\ohenno\\Documents\\Pays\\FRANCOPHONE.csv')
('MAJ de ANGLOPHONE, N° Id : 57 repertoire : '
 'C:\\Users\\ohenno\\Documents\\Pays\\ANGLOPHONE.csv')
('MAJ de ARABOPHONE, N° Id : 58 repertoire : '
 'C:\\Users\\ohenno\\Documents\\Pays\\ARABOPHONE.csv')
('MAJ de LUSOPHONE, N° Id : 59 repertoire : '
 'C:\\Users\\ohenno\\Documents\\Pays\\LUSOPHONE.csv')
"l'onglet AFRIQUE DU SUD du google sheet ne possède aucune données  "
('MAJ de ALGERIE, N° Id : 66 repertoire : '
 'C:\\Users\\ohenno\\Documents\\Pays\\ALGERIE.csv')
('MAJ de ANGOLA, N° Id : 317 repertoire : '
 'C:\\Users\\ohenno\\Documents\\Pays\\ANGOLA.csv')
('MAJ de BENIN, N° Id : 67 repertoire : '
 'C:\\Users\\ohenno\\Documents\\Pays\\BENIN.csv')
"l'onglet BOTSWANA du google sheet ne possède aucune données  "
('MAJ de BURKINA FASO, N° Id : 63 repertoire : '
 'C:\\Users\\ohenno\\Documents\\Pays\\BURKINA FASO.csv')
('MAJ de BURUNDI, N° Id : 105 repertoire : '
 'C:\\Users\\ohenno\\Doc